# Link Prediction in the Greek Web



In this notebook we are dealing with the problem of link prediction in graphs.Specifically we have a subset of the Greek web and we are trying to predict missing edges.
The approach we are going to follow is to deal with the problem as a binary classification task, where each edge is a candidate new edge for the graph. 
In the specific dataset there are 2041 nodes and 2683 edges and we are trying to predict the 453 edges that have been manually removed from the graph. We also have a dataset of different texts available from the hosts.

First we extract the raw text from the hosts

In [1]:
import os
import zipfile
import nltk
import pickle

if os.path.isfile('cache/processed_text.pickle'):
    with open('cache/processed_text.pickle', 'rb') as pfile:
        text_data = pickle.load(pfile)
        print "loaded from pickle"
else:
    filenames = os.listdir('dataset/hosts')
    raw_text = {}
    for zipfilename in filenames:
        with zipfile.ZipFile('dataset/hosts/'+zipfilename) as z:
            text = ""
            for filename in z.namelist():
                if not os.path.isdir(filename):
                    with z.open(filename) as f:
                        for line in f:
                            text += line.decode("utf-8").upper()
                            text += " "
            raw_text[zipfilename[:-4]] = text
    text_data = process_text(raw_text)
    with open('cache/processed_text.pickle', 'wb') as pfile:
        pickle.dump(text_data, pfile)

loaded from pickle


The function process_word is used for stemming as well as to remove tones from the greek text.

In [2]:
import stemmer as st
def process_word(word):
    """detone and stem word
    """
    new_word = []
    tones = { u'Ά' : u'A' , u'Έ': u'Ε', u'Ί': u'Ι', u'Ϊ': u'Ι',
             u'Ύ': u'Υ', u'Ϋ': u'Υ', u'Ό' : u'Ο', u'Ή': u'Η', u'Ώ': u'Ω'}
    for letter in word:
        try:
            new_word.append(tones[letter])
        except KeyError:
            new_word.append(letter)
    detoned = ''.join(l for l in new_word)
    return st.stem(detoned)
    

The function process_text is being used to remove stopwords, punctuation, numbers as well as use the above function for detoning and stemming.

In [3]:
import string
def process_text(data):
    with open('dataset/greekstopwords.txt', 'r') as fp:
        stopwords = []
        for line in fp:
            stopwords.append(line.strip().decode('utf-8').upper())
    for domain in data.keys():
        text = data[domain]
        # remove punctuation
        punctuation = set(string.punctuation)    
        doc = ''.join([w for w in text if w not in punctuation])
        # remove stopwords
        doc = [w for w in doc.split() if w not in stopwords]
        doc = [w for w in doc if not re.match(r"$\d+\W+|\b\d+\b|\W+\d+$", w)]
        doc = ' '.join(process_word(w) for w in doc)
        data[domain] = doc
    return data        


In [4]:
# assign each domain to the number. 
domain_number = {}
for i, domain in enumerate(text_data.keys()):
    domain_number[domain] = i

Afterwards we extract tfidf features from the websites. We choose 500 words as features and we compute the pairwise cosine similarity for all the websites. Below we also print those features.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vec = TfidfVectorizer(max_df=0.90, min_df=5, max_features=500,lowercase=False ,
                      analyzer = 'word')
x = vec.fit_transform(text_data.values())
for word in vec.get_feature_names():
    print word,
cosine = cosine_similarity(x)    

ABOUT ALL AND ARE AT ATHENS AΔΕΙ AΛΛ AΝΘΡΩΠ AΡΘΡ AΤΟΜ BLOG BY COMMENT COMMENTS CONTACT COOKIES COPYRIGHT DE DESIGN EMAIL FACEBOOK FM FOR FROM GOOGLE GREECE GREEK HOME HOT IN IS IT JULY LED LIFESTYLE LIKE LIVE MAY MEDIA MORE NEW NEWS NEWSLETTER NO OCTOBER OF ON ONLINE OR OUT POST POSTED POWERED RADIO READ RESERVED RIGHTS RSS SEARCH SEPTEMBER SHARE SITE THE THIS TO TOP TWEET TWITTER US VIDEO VIEW WEB WITH YOU YOUR ΑΓ ΑΓΟΡ ΑΓΟΡA ΑΓΩΝ ΑΘΗΝ ΑΘΛΗΤ ΑΘΛΗΤΙΣΜ ΑΚΟΛΟΥΘ ΑΚΟΜ ΑΛΛ ΑΛΛA ΑΝAΠΤΥΞ ΑΝΑΖΗΤΗΣ ΑΝΑΚΟΙΝΩΣ ΑΝΘΡΩΠ ΑΠ ΑΠAΝΤΗΣ ΑΠΟ ΑΠΟΣΤΟΛ ΑΠΟΤΕΛ ΑΠΟΤΕΛΕΣΜ ΑΠΟΦAΣ ΑΠΟΦΑΣ ΑΠΟΨ ΑΠΡΙΛ ΑΠΡΙΛΙ ΑΡΘΡ ΑΡΙΘΜ ΑΡΧ ΑΡΧΕΙ ΑΣΦAΛΕΙ ΑΤΤ ΑΥΓ ΑΥΓΟΥΣΤ ΑΥΤ ΑΥΤA ΑΥΤΟΚΙΝΗΤ ΑΦ ΒAΣ ΒΑΘΜΟΛΟΓ ΒΙΒΛ ΒΙΒΛΙ ΒΙΝΤΕ ΒΟΛ ΒΡ ΒΡΙΣΚ ΓΕΝ ΓΕΡΜΑΝ ΓΕΩΡΓΙ ΓΙAΝΝ ΓΙΑΤ ΓΙΝ ΓΙΩΡΓ ΓΝΩΣΤ ΓΟΝ ΓΡAΦ ΓΡΑΦΕΙ ΓΡΗΓΟΡ ΓΥΝΑΙΚ ΔΕ ΔΕΔΟΜΕΝ ΔΕΚ ΔΕΚΕΜΒΡ ΔΗΛΩΣ ΔΗΜ ΔΗΜΗΤΡ ΔΗΜΙΟΥΡΓ ΔΗΜΟΣ ΔΗΜΟΣΙ ΔΗΜΟΣΙΕΥΘ ΔΗΜΟΣΙΕΥΣ ΔΗΜΟΤ ΔΗΜΟΦΙΛ ΔΙAΒΑΣ ΔΙAΡΚΕΙ ΔΙAΦΟΡ ΔΙΑΒAΣ ΔΙΑΒAΣΤ ΔΙΑΓΩΝΙΣΜ ΔΙΑΤΡΟΦ ΔΙΑΦΗΜΙΣ ΔΙΑΧΕΙΡΙΣ ΔΙΕΘΝ ΔΙΕΥΘΥΝΣ ΔΙΚΑΙ ΔΙΚΑΙΩΜ ΔΙΚΤΥ ΔΙΝ ΔΙΟΙΚΗΣ ΔΡAΣ ΔΡΑΣΤΗΡΙΟΤ

In [6]:
# testing the text similarity of two websites
cosine[domain_number['news247.gr'], domain_number['newsit.gr']]

0.4863206743113806

In [7]:
from __future__ import division
import networkx as nx
import pprint
import random
import numpy as np
from scipy import sparse
from sklearn.model_selection import train_test_split


In [8]:
def text_similarity(src, dst):
    return cosine[domain_number[src], domain_number[dst]]  
    

### Topic Extraction and Document Clustering

In [9]:
# from time import time
# from sklearn.decomposition import NMF, LatentDirichletAllocation
# tfidf_vec = TfidfVectorizer(max_df=0.90, min_df=5, max_features=500,lowercase=False ,
#                           analyzer = 'word')
# tfidf = tfidf_vec.fit_transform(text_data.values())
# tf_vec = TfidfVectorizer(max_df=0.90, min_df=5, max_features=500,lowercase=False ,
#                           analyzer = 'word')
# tf = tf_vec.fit_transform(text_data.values())

In [10]:
# t0 = time()
# nmf = NMF(n_components=50, random_state=1,
#           alpha=.1, l1_ratio=.5).fit(tfidf)
# print("done in %0.3fs." % (time() - t0))

In [11]:
# def print_top_words(model, feature_names, n_top_words):
#     for topic_idx, topic in enumerate(model.components_):
#         print("Topic #%d:" % topic_idx)
#         print(" ".join([feature_names[i]
#                         for i in topic.argsort()[:-n_top_words - 1:-1]]))
#     print()

In [12]:
# print("\nTopics in NMF model:")
# tfidf_feature_names = tfidf_vec.get_feature_names()
# print_top_words(nmf, tfidf_feature_names, 10)

In [13]:
# lda = LatentDirichletAllocation(n_topics=605, max_iter=5,
#                                 learning_method='online',
#                                 learning_offset=50.,
#                                 random_state=0)
# t0 = time()
# lda.fit(tf)
# print("done in %0.3fs." % (time() - t0))

# print("\nTopics in LDA model:")
# tf_feature_names = tf_vec.get_feature_names()
# # print_top_words(lda, tf_feature_names, 10)


In [14]:
# dist = 1 -cosine
# from sklearn.cluster import KMeans

# num_clusters = 605
# km = KMeans(n_clusters=num_clusters)
# km.fit(tfidf)
# cluster_assignment = km.labels_.tolist()


In [15]:
# clusters = {i : [] for i in cluster_assignment}
# for node, cluster in enumerate(cluster_assignment):
#     clusters[cluster].append(G.nodes()[node])

    

In [16]:
# import pprint
# pprint.pprint(clusters,width=80)

## Graph

At first we create the directed graph from the 'edgelist.txt'.

In [3]:
import networkx as nx
import pprint

G = nx.read_edgelist('dataset/edgelist.txt', delimiter='\t', create_using=nx.DiGraph())
print len(G.nodes()), " number of nodes"
print len(G.edges()), " number of edges"

2041  number of nodes
2683  number of edges


In [18]:
# random.seed(1)
# edges = []
# with open('dataset/edgelist.txt', 'r')as fp:
#     for line in fp:
#         edges.append((line.split()[0], line.split()[1]))
# print len(edges)
# nodes = set([node for _tuple in edges for node in _tuple])
# print len(nodes)

## Train and Test Data

Training Set: For the training set we use 20760 edges that we are sure they do not exist (given in the 'non_existing_edges.txt' as the 1nd category and we use the 2683 edges from the graph which are the 2nd category for our classification problem.

Test Set: We use the 4160957 edges that do not exist in the graph. From those we remove the 20760 edges that we know that don't exist to end up with 4140197 candidate edges. 

For the edges available in the test set we will use a classifier to predict the class for each edge(0 or 1). We are interested in the probability that an edge exists. Afterwards we sort these probabilities and obtain the top 453 edges that are used for the submission.  
An alternate approach would be to use a number of random edges for the training set as nonexistent edges assuming that since the selective rate of such a set would be small that it would not affect as much the accuracy score.


In [4]:
#load the edges that do not exist
non_existent_edges = {}
with open('dataset/not_existing_edges.txt', 'r')as fp:
    for line in fp:
        non_existent_edges[((line.split()[0], line.split()[1]))] = 0
len(non_existent_edges)

20760

In [5]:
non_edges = [edge for edge in list(nx.non_edges(G)) if not non_existent_edges.has_key(edge)]
#4160957
len(non_edges)

4140197

In [21]:
# #new test with random edges
# #non_edges = [edge for edge in list(nx.non_edges(G))]
# non_existent_edges = {}
# random_numbers = [random.randint(0,len(non_edges)) for i in range(150000)]
# for i in random_numbers:
#     non_existent_edges[((non_edges[i][0], non_edges[i][1]))] = 0


## Extracting Features
We use networkx to help us extract useful features for each node of the graph

In [22]:
pagerank = nx.pagerank(G)
betweeness = nx.betweenness_centrality(G)
closeness = nx.closeness_centrality(G)
eigenvector = nx.eigenvector_centrality(G)
degree = nx.degree_centrality(G)
in_degree = G.in_degree()
out_degree = G.out_degree()
katz = nx.katz_centrality(G)
core_number = nx.core_number(G)
triangles = nx.triangles(G.to_undirected())

In [23]:
import graphsim as gs
simrank = gs.simrank(G)

Converge after 20 iterations (eps=0.000100).


In [24]:
import math
def adamic_adar(src, dst):
    score = 0
    common = list(set(G.neighbors(src)).intersection(G.neighbors(dst)))
    return sum(1 / math.log(G.degree(w))
                   for w in common)     


In [25]:
adamic_adar(u'news247.gr', u'contra.gr')

3.0020540579927957

In [26]:
un_g = G.to_undirected()

## Community Detection
We perform community detection on the graph using the louvain method. We obtain 605 number of communities. By examining them that some of them are really similar. For example they may belong to the same company, or they may have semantic similarity.
We use these communities to extract two features:  
Partition_common: declares the number of nodes from one's nodes neighborhood that exist in the same community as the other.  


In [6]:
import igraph as ig
import louvain_igraph as louvain
G_new = ig.Graph()
G_new.add_vertices(G.nodes())
G_new.add_edges(G.edges())

In [7]:
opt = louvain.Optimiser()
partition = opt.find_partition(graph=G_new,partition_class=louvain.SignificanceVertexPartition)

In [8]:
partition = list(partition)
partition_names = []
for com in partition:
    new_com = []
    for node_id in com:
        new_com.append(G_new.vs[node_id]['name'])
    partition_names.append(new_com)
# extract names
partitions = { node : [] for node in G.nodes()}
for com in partition_names:
    for node in com:
        partitions[node].extend(com)
print len(partition)

613


In [17]:
for i, com in enumerate(partition_names):
    if com:
        print i, " : ",
        for n in com:
            print n,",",
        print

0  :  politikanet.gr , katafylli.gr , skopelosonline.gr , inital.gr , arcadiasports.gr , bookone1.gr , oraiokastro24.gr , siriosfm.gr , sdeeth.gr , mypatra.gr , radioamfilochia.gr , ekriti.gr , paok-athens.gr , sarakis.gr , venceremos33.blogspot.gr , sportcyclades.gr , omospondiadaniolipton.gr , pireasnow.gr , radiomanos.gr , katanalotis.gr , nexusmanagementconsultants.gr , misitimi.gr , erassitexnikopodosfairo.blogspot.gr , pkteam.gr , betforthewinners.gr , eplski.gr , oragiaspor-dramas.gr , epskastorias.gr , frontpages.gr ,
1  :  tourist-guides.gr , mpass.gr , theacropolismuseum.gr , gbroofgarden.gr , cna.org.cy , grandebretagne.gr ,
2  :  fiat.gr , fiatprofessional.gr ,
3  :  left.gr , candianews.gr , papaioannou-giannis.net , epohi.gr , nonews-news.blogspot.gr , iskra.gr , pekp.gr , fisy.gr , pandiera.gr , fylosykis.gr , ekdohi.gr , alterthess.gr , rizospastis.gr ,
4  :  archelon.gr , 360photo.gr , lakepamvotis.gr , kerkini.gr , callisto.gr , siniparxi.gr , cob.gr ,
5  :  plantron.

In [31]:

def partition_common(src, dst):
    counter = 0
    for neigh in G.neighbors(src):
        if neigh in partitions[dst]:
            counter+=1
    for neigh in G.neighbors(dst):
        if neigh in partitions[src]:
            counter+=1
    return counter

def partition_check(src, dst):
    if src in partitions[dst]:
        return 1
    else:
        return 0
    
partition_common('news247.gr', 'ladylike.gr')


17

In [32]:
def second_neighbors(src, dst):
    """
    returns the number of common second level neighbors between two nodes"""
    level1_src = G.neighbors(src)
    level1_dst = G.neighbors(dst)
    score = 0
    level2_src = []
    level2_dst = []
    for w in level1_src:
        level2_src.extend(G.neighbors(w))
    for w in level1_dst:
        level2_dst.extend(G.neighbors(w))
    common = list(set(level2_src).intersection(level2_dst))
    return len(common)

In [33]:
second_neighbors('news247.gr', 'contra.gr')

15

In [34]:
from scipy import stats
print "pagerank", stats.describe(pagerank.values())
print "closeness", stats.describe(closeness.values())
print "betweeness", stats.describe(betweeness.values())
print "eigenvector", stats.describe(eigenvector.values())
print "text similarity", stats.describe(cosine)

pagerank DescribeResult(nobs=2041, minmax=(0.00029761511829001677, 0.019530823306958298), mean=0.00048995590396864249, variance=5.1484022288357726e-07, skewness=16.593681826353492, kurtosis=379.97825697924446)
closeness DescribeResult(nobs=2041, minmax=(0.0, 0.043706197338373103), mean=0.0011057427491154147, variance=1.0548295767353478e-05, skewness=6.840389761601526, kurtosis=62.844402531343476)
betweeness DescribeResult(nobs=2041, minmax=(0.0, 0.0013256810816528672), mean=5.1837164079498881e-06, variance=3.5261533531813181e-09, skewness=16.489608369782463, kurtosis=294.5096041003185)
eigenvector DescribeResult(nobs=2041, minmax=(0.0, 0.37735422231142191), mean=0.0017948109226848491, variance=0.00048697315309188523, skewness=13.386178390197925, kurtosis=183.2693369063938)
text similarity DescribeResult(nobs=2041, minmax=(array([ 0.,  0.,  0., ...,  0.,  0.,  0.]), array([ 1.,  1.,  1., ...,  1.,  1.,  1.])), mean=array([ 0.14081705,  0.11499577,  0.19841769, ...,  0.18956786,
        

In [35]:
nx.adamic_adar_index(un_g, [('news247.gr', 'contra.gr')]).next()

('news247.gr', 'contra.gr', 3.9275702562794277)

In [36]:
def feature_extraction(edge):
    """The function returns the feature vector of the edge
    """
    src, dst = edge
    f_vector = []
    f_vector.append(text_similarity(src, dst)) 
    f_vector.append(len(set(G.neighbors(src)).intersection(G.neighbors(dst))))
    f_vector.append(second_neighbors(src, dst))    
    f_vector.append(G.out_degree(src))
    f_vector.append(G.in_degree(dst))
    f_vector.append(pagerank[src])    
    f_vector.append(pagerank[dst])
    f_vector.append(eigenvector[src])
    f_vector.append(eigenvector[dst])
    f_vector.append(betweeness[src])
    f_vector.append(betweeness[dst])
    f_vector.append(closeness[src])
    f_vector.append(closeness[dst])
    f_vector.append(katz[src])
    f_vector.append(katz[dst])
    f_vector.append(core_number[src])
    f_vector.append(core_number[dst])
    f_vector.append(triangles[src])
    f_vector.append(triangles[dst])
    f_vector.append(simrank[domain_number[src], domain_number[dst]])
    f_vector.append(nx.adamic_adar_index(un_g, [(src, dst)]).next()[2])
    f_vector.append(nx.jaccard_coefficient(un_g, [(src, dst)]).next()[2])
    f_vector.append(nx.preferential_attachment(un_g, [(src, dst)]).next()[2])
    f_vector.append(nx.resource_allocation_index(un_g, [(src, dst)]).next()[2])
    f_vector.append(partition_check(src, dst))
    f_vector.append(partition_common(src, dst))
    if G.has_edge(dst,src):
        f_vector.append(1)
    else:
        f_vector.append(0) 
    return f_vector

feature_names = ["text","#common_neighbors","#_of_second_neighbors", "G.out_degree(src)","G.in_degree(dst)",
                 "pagerank[src]","pagerank[dst]","eigenvector[src]","eigenvector[dst]","betweeness[src]",
                 "betweeness[src]","betweeness]dst","closeness[src]","closeness[dst]","katz[src]","katz[dst]",
                 "core_number[src]","core_number[dst]","triangles[src]","triangles[dst]","simrank", "adamic_adar",
                 "jaccard_coefficient","preferential_attachment","resource_allocation_index",
                 "partition_check", "opposite_edge"]

# def feature_extraction(edge):
#     src, dst = edge
#     f_vector = []
#     f_vector.append(text_similarity(src, dst))
#     f_vector.append(G.out_degree(src))
#     f_vector.append(G.in_degree(dst))
#     f_vector.append(pagerank[src])
#     f_vector.append(eigenvector[src])
#     f_vector.append(eigenvector[dst])
#     f_vector.append(pagerank[dst])
#     f_vector.append(betweeness[src])
#     f_vector.append(betweeness[dst])
#     f_vector.append(closeness[dst])
#     f_vector.append(closeness[src])
#     f_vector.append(adamic_adar(src, dst))
#     f_vector.append(second_neighbors(src, dst))
#     f_vector.append(partition_check(src, dst))
#     if G.has_edge(dst,src):
#         f_vector.append(1)
#     else:
#         f_vector.append(0)    
#     return f_vector
    
    
# def feature_extraction(edge):
#     np.random.seed(seed)
#     src, dst = edge
#     feature_names = ["text", "second_neighbors","G.out_degree(src)","G.in_degree(dst)","pagerank[src]"
#                      , "eigenvector[dst]","betweeness[src]","closeness[dst]","preferential_attachment",
#              "resource_allocation_inde","partition_check"]
#     f_vector = []
#     f_vector.append(text_similarity(src, dst))
#     #f_vector.append(len(set(G.neighbors(src)).intersection(G.neighbors(dst))))
#     f_vector.append(second_neighbors(src, dst))    
#     f_vector.append(G.out_degree(src))
#     f_vector.append(G.in_degree(dst))
#     f_vector.append(pagerank[src])    
#     #f_vector.append(pagerank[dst])
#     #f_vector.append(eigenvector[src])
#     f_vector.append(eigenvector[dst])
#     f_vector.append(betweeness[src])
#     #f_vector.append(betweeness[dst])
#     f_vector.append(closeness[dst])
#     #f_vector.append(closeness[src])
#     #f_vector.append(adamic_adar2(src, dst))
#     #f_vector.append(nx.adamic_adar_index(un_g, [(src, dst)]).next()[2])
#     #f_vector.append(nx.jaccard_coefficient(un_g, [(src, dst)]).next()[2])
#     f_vector.append(nx.preferential_attachment(un_g, [(src, dst)]).next()[2])
#     f_vector.append(nx.resource_allocation_index(un_g, [(src, dst)]).next()[2])
#     f_vector.append(partition_check(src, dst))
#     #return np.random.choice(f_vector,num),seed, names
#     return f_vector



In [37]:
seed = np.random.randint(1,4000000) 

In [2]:
   
#vector,seed,names = feature_extraction(('news247.gr','contra.gr'),5 ,seed)
vector = feature_extraction(('news247.gr','contra.gr'))
if len(feature_names) != len(vector):
    raise Exception
print vector

NameError: name 'feature_extraction' is not defined

In [40]:
X_train = []
y_train = []
for edge in G.edges():
    X_train.append(feature_extraction(edge))
    y_train.append(1)
for edge in non_existent_edges:
    X_train.append(feature_extraction(edge))
    y_train.append(0)


In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_predict, y_train, y_test = train_test_split( X_train, y_train, test_size=0.10, random_state=42)

In [41]:
X_predict = []
for edge in non_edges:
    X_predict.append(feature_extraction(edge))

In [42]:
X_train_n = np.array(X_train)
y_train_n = np.array(y_train)
X_predict_n = np.array(X_predict)


In [43]:
# scale features
from sklearn import preprocessing
# from sklearn import preprocessing
X_train_scaled = preprocessing.scale(X_train_n)
X_predict_scaled = preprocessing.scale(X_predict_n)

In [44]:
# PCA didnt work
# from sklearn.decomposition import PCA
# pca = PCA(n_components=10)
# X_train_n = pca.fit_transform(X_train)
# X_predict_n = pca.fit_transform(X_predict)

In [45]:
np.save("cache/X_train_new.npy", X_train_n)
np.save("cache/y_train_new.npy", y_train_n)
np.save("cache/X_test_new.npy", X_predict_n)

In [46]:
print X_train_n.shape
print y_train_n.shape
print X_predict_n.shape

(23443, 27)
(23443,)
(4140197, 27)


## Feature Selection

There are four approaches that can be followed for feature selection:  

1. Do manual feature selection using the code below. We print some statistics about each feature to see if some of them have low variance. Afterwards we also calculate the pearson correlation between each pair of features and choose to remove one of the features in each pair that shows a high correlation. High correlation between two features can make our model really unstable.  
2. Use the feature_selection package available in scikit learn to do feature selection.  
3. Use a model that provides feature importance (eg Random Forests).  
4. Do no feature selection and use all features in a neural network of a tree based model  

We experiment with all the choices we have, but doing feature selection is important and produces a more robust model than using all avalable features.

In [ ]:
from scipy import stats
from tabulate import tabulate


#print tabulate(range(0,18), feature_names, tablefmt="grid")
feats = {}
for i, name in enumerate(feature_names):
    feats[name]={}
    feats[name]["mean"]=stats.describe(X_train_n[:,i]).mean
    feats[name]["min_max"]=stats.describe(X_train_n[:,i]).minmax
    feats[name]["variance"]=stats.describe(X_train_n[:,i]).variance

for name, _dict in feats.iteritems():
    print name, tabulate(_dict.items())

from itertools import combinations
f_combs = list(combinations(range(len(feature_names)), 2))

from scipy.stats import pearsonr
pearson_corr = np.zeros((len(f_combs),len(f_combs)))
for f1, f2 in f_combs:
    pearson_corr[f1, f2] = pearsonr(X_train_n[:2683,f1], X_train_n[:2683,f2])[0]
    pearson_corr[f2, f1] = pearson_corr[f1, f2] 


    

#for f1, f2 in f_combs:
    #print feature_names[f1], feature_names[f2], pearson_corr[f1, f2]
for i, name in enumerate(feature_names):
    print name, feature_names[np.argmax(pearson_corr[i, :])] ,pearson_corr[i,np.argmax(pearson_corr[i, :])]

In [ ]:
# we save the numpy arrays to be able to run classification algorithms 
# again without running all the above processes
np.save("cache/X_train.npy", X_train_n)
np.save("cache/y_train.npy", y_train_n)
np.save("cache/X_test.npy", X_predict_n)

## Classification

For classication we test several cases. At first we use logistic regression, SVM and a neural network classifier.  
As we have rather small data opposed to the candidate space (2683 true edges opposed to ~4m edges) we have to be very careful with regularization because most models tend to overfit the training data.  
Afterward we experiment with Random Forests as well as ensemble methods such as Gradient boosting, XGBoost which are methods that have gained much attendance lately, especially due to their high performance in Kaggle contests.  
However such models require very good tuning because of the high parameter space they have, so we perform Grid Search to find the optimal parameters for the Random Forest model as well as XGBoost with all the candidate features. Unfortunately because of the computational complexity of such a task we did not perform Grid Search to models with different features.
In the end we also run a voting classifier with the best tuning for XGBoost and Random Forest.  
We want to use classification algorithms that implement the predict_proba method, because we are interested in the probability that an edge will exist in our graph. Because of the small dataset and the overfitting that happens we must balance the classifiers uncertainty. 

The highest (reproducible) score (10,15%) was obtained using XGBoost and the following feature vector function:

def feature_extraction(edge):  
        src, dst = edge  
        f_vector = []  
        f_vector.append(text_similarity(src, dst))  
    f_vector.append(pagerank[src])  
    f_vector.append(eigenvector[src])  
    f_vector.append(eigenvector[dst])  
    f_vector.append(pagerank[dst])  
    f_vector.append(betweeness[src])  
    f_vector.append(betweeness[dst])  
    f_vector.append(closeness[dst])  
    f_vector.append(closeness[src])  
    f_vector.append(adamic_adar(src, dst))  
    f_vector.append(partition_check(src, dst))  
    return f_vector  
    
We also scored a 10.31% but it was with a Random Forest without saving the random seed.

In [ ]:
# if we have not ran everything above we can just run this to load the Train/Test data.
X_train_n = np.load('cache/X_train.npy')
y_train_n = np.load('cache/y_train.npy')
X_predict_n = np.load('cache/X_test.npy')

In [ ]:
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression(C= 0.00001)
reg.fit(X_train_scaled, y_train_n)
pred_reg = reg.predict_proba(X_predict_scaled)
probs_reg = []
for t in pred_reg:
    probs_reg.append(t[1])
indices = sorted(range(len(probs_reg)), key=lambda k: probs_reg[k])[::-1][:453]
print [probs_reg[indices[i]] for i in range(len(indices))]
predicted_edges_reg = []
for i in range(len(indices)):
    predicted_edges_reg.append(non_edges[indices[i]])

with open('predicted_edges_logReg.txt', 'w') as fp:
    for site_1, site_2 in predicted_edges_reg:
        fp.write(site_1+'\t'+site_2+'\n')

In [47]:
from sklearn.svm import SVC
svm = SVC(probability=True, C=0.00001)
svm.fit(X_train_scaled, y_train_n)
pred_svm = svm.predict_proba(X_predict_scaled)
probs_svm = []
for t in pred_svm:
    probs_svm.append(t[1])
indices = sorted(range(len(probs_svm)), key=lambda k: probs_svm[k])[::-1][:453]
print [probs_svm[indices[i]] for i in range(len(indices))]
predicted_edges_svm = []
for i in range(len(indices)):
    predicted_edges_svm.append(non_edges[indices[i]])

with open('predicted_edges_svm.txt', 'w') as fp:
    for site_1, site_2 in predicted_edges_svm:
        fp.write(site_1+'\t'+site_2+'\n')

[0.97901864078349621, 0.97900787000814349, 0.97882682877148808, 0.97840794708899659, 0.97823033746886889, 0.97810362026197262, 0.97808123062086427, 0.97786971299010994, 0.97773346557348706, 0.97762762585553464, 0.97750833549185578, 0.97740563756198873, 0.97724291215797465, 0.9767270076136827, 0.97662736365929903, 0.97657315537475375, 0.97656403267067127, 0.97638297734625878, 0.976237173163782, 0.97620190113319327, 0.97612054867394571, 0.97609972633630671, 0.97606677149329202, 0.97604964689049878, 0.97594805038454757, 0.97591506552841489, 0.97588742658462413, 0.97569148829363928, 0.97567270131228134, 0.97565904456945485, 0.9755847453796529, 0.9755712860807827, 0.97550156016625644, 0.97549143585434728, 0.97547221850980037, 0.97540926469948397, 0.9753597840774102, 0.97534116264717818, 0.97526771951401292, 0.97506885972940593, 0.97499665661450308, 0.97497378454650097, 0.97494297191504908, 0.97494109821434149, 0.97492532285980538, 0.97491389806070838, 0.97488353491760027, 0.9748749682105721

In [54]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(activation = 'logistic',solver='sgd', alpha=0.2, hidden_layer_sizes=(80,40), random_state=1)

mlp.fit(X_train_scaled, y_train_n) 

pred_mlp = mlp.predict_proba(X_predict_scaled)

probs_mlp = []
for t in pred_mlp:
    probs_mlp.append(t[1])
indices = sorted(range(len(probs_mlp)), key=lambda k: probs_mlp[k])[::-1][:453]
print [probs_mlp[indices[i]] for i in range(len(indices))]

predicted_edges_mlp = []
for i in range(len(indices)):
    predicted_edges_mlp.append(non_edges[indices[i]])

with open('predicted_edges_mlp.txt', 'w') as fp:
    for site_1, site_2 in predicted_edges_mlp:
        fp.write(site_1+'\t'+site_2+'\n')

[0.99572798653903372, 0.99572683582710653, 0.99572408430148485, 0.99571609751201484, 0.99570666296802701, 0.99570116739280645, 0.99569966061595672, 0.99569700186883647, 0.99569661753706207, 0.99568995443535835, 0.99568982096971548, 0.99568886473806872, 0.99568692254321234, 0.99568446010303457, 0.9956829321210825, 0.99568288307922515, 0.99568260987182033, 0.99567854601430028, 0.99567844418002593, 0.99567747196495182, 0.99567730348051042, 0.99567644968647839, 0.99567533676972697, 0.99567438943883479, 0.99567163199951292, 0.995671352394361, 0.99567015420532512, 0.99566837887559689, 0.99566824913641216, 0.99566728402721738, 0.99566716139559008, 0.99566675212841027, 0.99566669088915782, 0.99566591003979377, 0.99566548038504021, 0.9956638349073178, 0.9956635966913584, 0.99566358957283962, 0.99566321020750648, 0.99566314071243756, 0.99566250880255858, 0.99566207634205084, 0.99566194308706335, 0.99566150627167083, 0.99566091901115195, 0.99565916003407151, 0.99565882351286128, 0.995658031862424

In [16]:
import numpy as np
import random
X_train_n = np.load('cache/X_train.npy')
y_train_n = np.load('cache/y_train.npy')
X_predict_n = np.load('cache/X_test.npy')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
np.random.seed(10)
seed = np.random.randint(1,4000000)
print seed
#{'bootstrap': False, 'min_samples_leaf': 3, 'min_samples_split': 10, 'criterion': 'gini', 
                #'max_features': 4, 'max_depth': 8}
    #'bootstrap': True, 'min_samples_leaf': 3, 'min_samples_split': 2, 'criterion': 'entropy',
    #'max_features': 8, 'max_depth': 8}
#forest = RandomForestClassifier(random_state=seed,n_estimators=80, min_samples_split=10,
                                #max_features=4, max_depth=8, min_samples_leaf=3,criterion='gini')
# Parameters: {'bootstrap': True, 'min_samples_leaf': 3, 'min_samples_split': 2, 'criterion': 'gini',
#              'max_features': 3, 'max_depth': 8}

forest = RandomForestClassifier(random_state=seed, n_estimators=120, min_samples_split=3, min_samples_leaf=3,
                                bootstrap=False,  max_features=3, max_depth=8, criterion='gini')

forest.fit(X_train_n, y_train_n)
pred_forest = forest.predict_proba(X_predict_n)
probs_forest = []
for t in pred_forest:
    probs_forest.append(t[1])
indices = sorted(range(len(probs_forest)), key=lambda k: probs_forest[k])[::-1][:453]
predicted_edges_forest = []
for i in range(len(indices)):
        predicted_edges_forest.append(non_edges[indices[i]])
print [probs_forest[indices[i]] for i in range(len(indices))]
with open('predicted_edges_forest.txt', 'w') as fp:
    for site_1, site_2 in predicted_edges_forest:
        fp.write(site_1+'\t'+site_2+'\n')
f_ind = np.argsort(forest.feature_importances_)[::-1]
for i in f_ind:
    print "\n",feature_names[i],forest.feature_importances_[i]

3491082


In [1]:
import xgboost
xgb = xgboost.XGBClassifier()
xgb.fit(X_train_n, y_train_n)
pred_xgb = xgb.predict_proba(X_predict_n)

f_ind = np.argsort(xgb.feature_importances_)[::-1]
for i in f_ind:
    print feature_names[i],xgb.feature_importances_[i]

probs_xgb = []
for t in pred_xgb:
    probs_xgb.append(t[1])
indices = sorted(range(len(probs_xgb)), key=lambda k: probs_xgb[k])[::-1][:453]
predicted_edges_xgb = []
counter=0
for i in range(len(indices)):
    predicted_edges_xgb.append(non_edges[indices[i]])
print [probs_xgb[indices[i]] for i in range(len(indices))]
with open('predicted_edges_xgb.txt', 'w') as fp:
    for site_1, site_2 in predicted_edges_xgb:
        fp.write(site_1+'\t'+site_2+'\n')

/home/lias/PycharmProjects/link-prediction/venv/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


NameError: name 'X_train_n' is not defined

## Gradient Boosting

In [28]:
class CompatClassifier(RandomForestClassifier):
   def predict(self, X):
       return self.predict_proba(X)[:, 1][:,np.newaxis]
seed = random.randint(1, 400000)
print seed   
#278753

base_estimator = CompatClassifier(random_state=3808056, n_estimators=10, min_samples_split=3, min_samples_leaf=3,
                                bootstrap=False,  max_features=3, max_depth=8, criterion='gini')
#min_samples_split=10,
                                #max_features=4, max_depth=8, min_samples_leaf=3,criterion='gini'
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(random_state=3808056,init=base_estimator,n_estimators=150)

gbc.fit(X_train_n,y_train_n)
preds_gbc = gbc.predict_proba(X_predict_n)
probs_gbc = []
for t in preds_gbc:
    probs_gbc.append(t[1])
indices = sorted(range(len(probs_gbc)), key=lambda k: probs_gbc[k])[::-1][:453]
predicted_edges_gbc = []
for i in range(len(indices)):
        predicted_edges_gbc.append(non_edges[indices[i]])
print [probs_gbc[indices[i]] for i in range(len(indices))]
with open('predicted_edges_gbc.txt', 'w') as fp:
    for site_1, site_2 in predicted_edges_gbc:
        fp.write(site_1+'\t'+site_2+'\n')
f_ind = np.argsort(gbc.feature_importances_)[::-1]
for i in f_ind:
    print "\n",feature_names[i],gbc.feature_importances_[i]

231237
[0.99990616443726688, 0.99989693841848748, 0.9998922135588949, 0.99988683938769096, 0.99987787698348329, 0.99987527473687998, 0.99986852023662576, 0.99986574054235766, 0.99986418456194326, 0.99986177513969532, 0.99986072271133863, 0.99985814661859973, 0.99985128516004773, 0.99984314385779904, 0.99984074948899948, 0.99983943596107994, 0.99983458566184436, 0.99982975321975842, 0.99981782635897509, 0.99981782635897509, 0.99980833273367997, 0.99980272558311034, 0.99979805495144802, 0.99979786179723296, 0.99979750946688462, 0.99979728275942537, 0.99979706372470112, 0.99979704778274203, 0.99979115408838937, 0.9997877920283178, 0.99978345264977553, 0.99978236365253526, 0.9997782472990473, 0.99977650717199418, 0.99977091796561324, 0.99977026501521715, 0.99976628134892487, 0.99976570637735251, 0.99976212216886828, 0.99975728058394364, 0.99975464400733816, 0.99975206550359208, 0.99975199676534676, 0.99975065085003945, 0.99974980430906424, 0.99974702336200116, 0.99974622879314978, 0.999744

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint as sp_randint
from time import time
# build a classifier
clf = RandomForestClassifier(n_estimators=50, n_jobs=4)
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

# use a full grid over all parameters
param_grid = {"max_depth": range(3, 9),
              "max_features": range(2, 9),
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X_train_n, y_train_n)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)
with open('grid_forest.pickle', 'wb') as pfile:
        pickle.dump(grid_search.cv_results_, pfile)

In [ ]:
import xgboost
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint as sp_randint
from time import time 
# build a classifier
clf = xgboost.XGBClassifier()
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
"""(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)>"""
# use a full grid over all parameters
param_grid = {
    "learning_rate": np.arange(0.15, 0.25, 0.05),
    "max_depth": range(4, 6),
    "max_delta_step": np.arange(0, 0.15, 0.05),
    "gamma": np.arange(0, 0.15, 0.05),
    "max_delta_step": np.arange(0, 0.15, 0.05),
    "subsample": np.arange(0.7, 1, 0.1),
    "min_child_weight": np.arange(0.8, 1.20, 0.1),}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X_train_n, y_train_n)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)
with open('grid_xgb.pickle', 'wb') as pfile:
        pickle.dump(grid_search.cv_results_, pfile)

In [ ]:
from sklearn.ensemble import VotingClassifier
xgb = xgboost.XGBClassifier()
forest = RandomForestClassifier(random_state=seed,n_estimators=80, max_features=6, max_depth=7, min_samples_leaf=3,oob_score=True)
eclf1 = VotingClassifier(estimators=[('forest', forest), ('xgb', xgb)], voting='soft')

eclf1 = eclf1.fit(X_train_n, y_train_n)
voting_pred = eclf1.predict_proba(X_predict_n)

probs_vote = []
for t in voting_pred:
    probs_vote.append(t[1])
indices = sorted(range(len(probs_vote)), key=lambda k: probs_vote[k])[::-1][:453]
predicted_edges_vote = []
for i in range(len(indices)):
    predicted_edges_vote.append(non_edges[indices[i]])
print [probs_vote[indices[i]] for i in range(len(indices))]
with open('predicted_edges_voting.txt', 'w') as fp:
    for site_1, site_2 in predicted_edges_vote:
        fp.write(site_1+'\t'+site_2+'\n')

## Mining data from external sources (cheating?)

Because of the public dataset we could not stand but try to obtain information from the data source (the Web!). So we downloaded/parsed the homepage of each host and obtained the links to other webpages that exist in the dataset. The search was not exhaustive(we set a strict response timeout) because it was supposed to be just a test, mostly trying to see how high such an (illegal) method would score. I believe that scoring 23% accuracy with this way proved the problems difficulty.

In [ ]:
import ssl
import requests
import re
from urlparse import urlparse
from bs4 import BeautifulSoup
def get_links(url):
    try:
        resp = requests.get('http://'+url,verify=False, timeout=(5,20))
    except Exception as e:
        print e
        return []
    html = resp.text
    soup = BeautifulSoup(html, "html5lib")
    links = soup.findAll("a")
    reg = '^(www.)'
    try:
        links = [re.sub(reg, '', urlparse(link.get('href')).netloc) for link in links]
    except AttributeError:
        pass
    return list(set([link for link in links if G.has_node(link) and not G.has_edge(url, link) and link !=url] ))

if os.path.isfile('cache/crawled_data.pickle'):
    with open('cache/crawled_data.pickle', 'rb') as pfile:
        text_data = pickle.load(pfile)
        print "loaded from pickle"
else:    
    missing = {}
    for node in G.nodes():    
        # Start the timer. Once 5 seconds are over, a SIGALRM signal is sent.
        signal.alarm(25)    
        # This try/except loop ensures that 
        #   you'll catch TimeoutException when it's sent.
        try:
            print node,
            missing[node]=get_links(node) # Whatever your function that might hang
        except TimeoutException:
            continue # continue the for loop if function A takes more than 5 second
        else:
            # Reset the alarm
            signal.alarm(0)
    with open('cache/crawled_data.pickle', 'wb') as pfile:
        pickle.dump(missing, pfile)

counter  = 0
for key,value in missing.iteritems():
    if len(value)>0:
        counter+=len(value)
print counter

crawled = []
for src, links in missing.iteritems():
    for dst in links:
        crawled.append((src,dst))


with open('predicted_edges_crawled.txt', 'w') as fp:
    for site_1, site_2 in crawled:
        fp.write(site_1+'\t'+site_2+'\n')
    for site_1, site_2 in predicted_edges_xgb:
        if (site_1,site_2) not in crawled:
            fp.write(site_1+'\t'+site_2+'\n')